In [2]:
key = "k_frwsnx5q"

In [3]:
import numpy as np
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.datasets import load_digits
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
import tensorflow as tf
from keras.optimizer_v2.adam import Adam
from sklearn.ensemble import RandomForestClassifier
import gc
import movie_scraper
import ast
import csv
import time

Init Plugin
Init Graph Optimizer
Init Kernel


In [4]:
def to_final_class_vector(i):
    indice = np.where(i == np.amax(i))
    i2 = np.zeros(5)
    i2[indice[0][0]] = 1
    if indice[0][0] == 0:
        print("Great")
    elif indice[0][0] == 1:
        print("Good")
    elif indice[0][0] == 2:
        print("Okay")
    elif indice[0][0] == 3:
        print("Bad")
    else:
        print("Awful")
    return i2

Webscraping

In [5]:
f = open("testing_files/test_names.txt", "r")
names = f.readlines()
w0 = open("testing_files/test.txt", "a")

for i in range (len(names)):
    #15 API Calls per film

    time.sleep(0.5)
    movie_name = names[i]
    movie_id = movie_scraper.get_movie_id(key, movie_name)
    movie_rating = movie_scraper.get_ratings(key, movie_id, movie_name)
    movie_actors, movie_actors_ids = movie_scraper.get_actors(key, movie_id, movie_name)
    movie_num_wins, movie_num_nominations = movie_scraper.get_movie_awards(key, movie_id, movie_name)
    movie_actors_awards, cum_actors_oscars, cum_actors_wins, cum_actors_nominations = movie_scraper.get_actors_awards(key, movie_actors_ids, movie_actors)
    movie_demographics0, movie_demographics1, movie_demographics2, movie_demographics3, movie_demographics4 = movie_scraper.get_gender_and_age_demographics(key, movie_id, movie_name)

    data = (movie_name, movie_id, movie_rating[0], 
            movie_actors, movie_actors_ids, movie_num_wins, 
            movie_num_nominations, movie_actors_awards, 
            cum_actors_oscars, cum_actors_wins, cum_actors_nominations, 
            movie_demographics0, movie_demographics1, movie_demographics2, movie_demographics3, movie_demographics4)
    
    if not all(data):
        print("FAILED: " + str(movie_name))


    else:
        print("SUCESS: " + str(movie_name))

        data = {"movie name": movie_name, 
                "movie id": movie_id, 
                "movie rating": movie_rating[0], #y
                "movie num awards": movie_num_wins,
                "movie num nominations": movie_num_nominations,
                "percent male reviewers": movie_demographics0, #x (0.0-1.0)
                "percent female reviewers": movie_demographics1, #x (0.0-1.0)
                "percent under 30 reviewers": movie_demographics2, #x (0.0-1.0)
                "percent 30-45 reviewers": movie_demographics3, #x (0.0-1.0)
                "percent over 45 reviewers": movie_demographics4, #x (0.0-1.0)
                "movie actor awards": movie_actors_awards,
                "cumulative actor oscars": cum_actors_oscars, #x (a reasonable integer with no extreme values)
                "cumulative actor wins": cum_actors_wins, #x (a reasonable integer with no extreme values)
                "cumulative actor nominations": cum_actors_nominations} #x (a reasonable integer with no extreme values)

        w0.write((str(data)))
        w0.write('\n')

w0.close()
f.close()

SUCESS: Pulp Fiction

SUCESS: The Amazing Spider-Man 2

SUCESS: The Shawshank Redemption



Creating CSV datafiles

In [6]:
f = open("testing_files/test.txt", "r")
dictionaries = f.readlines()

for i in range(len(dictionaries)):
    dictionaries[i] = (dictionaries[i]).strip()

def data():
    
    for j in range(len(dictionaries)):
        try: 
            if len(dictionaries[j]) != 0:
                elements = ast.literal_eval((dictionaries[j]))

                mr  = elements["movie rating"]
                pmr = elements["percent male reviewers"]
                pfr = elements["percent female reviewers"]
                put = elements["percent under 30 reviewers"]
                ptf = elements["percent 30-45 reviewers"]
                pof = elements["percent over 45 reviewers"]
                cao = elements["cumulative actor oscars"]
                caw = elements["cumulative actor wins"]
                can = elements["cumulative actor nominations"]

            x_val = [pmr,pfr,put,ptf,pof,cao,caw,can]
            y_val_continuous = [float(mr)]

            if y_val_continuous[0] >= 8.0:
                y_val_classification = [1, 0, 0, 0, 0]
            elif y_val_continuous[0] >= 7.0:
                y_val_classification = [0, 1, 0, 0, 0]
            elif y_val_continuous[0] >= 6.0:
                y_val_classification = [0, 0, 1, 0, 0]
            elif y_val_continuous[0] >= 3.0:
                y_val_classification = [0, 0, 0, 1, 0]
            else:
                y_val_classification = [0, 0, 0, 0, 1]

            with open('testing_files/test_x.csv', 'a', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow(x_val)

            f.close()

            with open('testing_files/test_y_continuous.csv', 'a', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow(y_val_continuous)

            f.close()

            with open('testing_files/test_y_classification.csv', 'a', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow(y_val_classification)

            f.close()

            print(f'x features: {x_val}')
            print(f'y value for regression: {y_val_continuous}')
            print(f'y value for classification: {y_val_classification}')

        except Exception as e:
            print(e)
            print("Failed at row " + str(j))

data()


x features: [0.84, 0.16, 0.22, 0.61, 0.17, 10, 168, 283]
y value for regression: [8.9]
y value for classification: [1, 0, 0, 0, 0]
x features: [0.86, 0.14, 0.32, 0.56, 0.12, 8, 286, 616]
y value for regression: [6.6]
y value for classification: [0, 0, 1, 0, 0]
x features: [0.83, 0.17, 0.23, 0.61, 0.16, 6, 128, 158]
y value for regression: [9.3]
y value for classification: [1, 0, 0, 0, 0]


In [7]:
actors = ["Natalie Portman", "Andrew Garfield", "Will Smith", "Chris Rock", "Ben Affleck", "Gal Gadot", "Chris Evans", "Kevin James", "Chris Pine", "Kevin Hart"]
percentMaleAudience = 0.8
percent30under = 0.60
percent30_45 = 0.30
percent45over = 0.10

In [8]:
feature_vector = movie_scraper.to_Feature_Vector(actors, percentMaleAudience, percent30under, percent30_45, percent45over)
print(feature_vector)

[0.03079610305453737, 6.034778058286225, -3.97993563087433, -1.3319108266550153, 1.5427372012928036, 3.0937267223342766, 3.965337133506138]


Regression Prediction

In [9]:
sample_model = keras.models.load_model('saved_models/nn/regression/regression_model_fold_1.h5')

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-05-04 18:52:31.911510: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-04 18:52:31.911651: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
with tf.device('/device:GPU:0'):
    pred = sample_model.predict([feature_vector])
print(pred)
print(type(pred))

2022-05-04 18:52:41.746127: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-04 18:52:41.746304: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-04 18:52:41.781563: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


[[7.9171705]]
<class 'numpy.ndarray'>


CLassification Prediction

In [11]:
sample_model2 = keras.models.load_model('saved_models/nn/classification/classification_model_fold_1.h5')

In [12]:
with tf.device('/device:GPU:0'):
    pred2 = sample_model2.predict([feature_vector])[0]
print(pred2)
print(to_final_class_vector(pred2))


2022-05-04 18:53:01.624151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


[0.08040649 0.2495852  0.4207601  0.24214728 0.00710093]
Okay
[0. 0. 1. 0. 0.]


In [ ]:
preds = []
for i in range(10):
    sample_model = keras.models.load_model(f'saved_models/nn/regression/regression_model_fold_{i}.h5')
    pred = sample_model.predict([[0.406931714,1.071644936,-0.004680542,-0.717662002,0.880286532,0.759274252,0.483845934]])[0]
    preds.append(pred)
    del sample_model
    del pred
    gc.collect
    keras.backend.clear_session()
preds = np.array(preds)
print(preds)

In [ ]:
def ensemble_nn_prediction(models, regression, X):
    pred = None
    preds = []
    if regression:
        for model in models:
            predY = model.predict(X)
            preds.append(predY)
        avg = np.average(np.array(preds))
        pred = avg
    else:
        for model in models:
            predY = model.predict(X)
            preds.append(predY[0])
        preds = np.array(pred)
        pred = np.sum(preds, axis=0)
        pred = to_final_class_vector(pred)

    return pred